# Rf 5 0 1_Simultaneouspdf
Organisation and simultaneous fits: using simultaneous p.d.f.s to describe simultaneous fits to multiple datasets



**Author:** 07/2008 - Wouter Verkerke  
<i><small>This notebook tutorial was automatically generated with <a href= "https://github.com/root-project/root/blob/master/documentation/doxygen/converttonotebook.py">ROOTBOOK-izer</a> from the macro found in the ROOT repository  on Thursday, August 29, 2019 at 02:54 AM.</small></i>

In [1]:
%%cpp -d
#include "RooRealVar.h"
#include "RooDataSet.h"
#include "RooGaussian.h"
#include "RooConstVar.h"
#include "RooChebychev.h"
#include "RooAddPdf.h"
#include "RooSimultaneous.h"
#include "RooCategory.h"
#include "TCanvas.h"
#include "TAxis.h"
#include "RooPlot.h"

In [2]:
%%cpp -d
// This is a workaround to make sure the namespace is used inside functions
using namespace RooFit;

Create model for physics sample
 -------------------------------------------------------------

Create observables

In [3]:
RooRealVar x("x", "x", -8, 8);


RooFit v3.60 -- Developed by Wouter Verkerke and David Kirkby 
                Copyright (C) 2000-2013 NIKHEF, University of California & Stanford University
                All rights reserved, please read http://roofit.sourceforge.net/license.txt



Construct signal pdf

In [4]:
RooRealVar mean("mean", "mean", 0, -8, 8);
RooRealVar sigma("sigma", "sigma", 0.3, 0.1, 10);
RooGaussian gx("gx", "gx", x, mean, sigma);

Construct background pdf

In [5]:
RooRealVar a0("a0", "a0", -0.1, -1, 1);
RooRealVar a1("a1", "a1", 0.004, -1, 1);
RooChebychev px("px", "px", x, RooArgSet(a0, a1));

Construct composite pdf

In [6]:
RooRealVar f("f", "f", 0.2, 0., 1.);
RooAddPdf model("model", "model", RooArgList(gx, px), f);

Create model for control sample
 --------------------------------------------------------------

Construct signal pdf.
 NOTE that sigma is shared with the signal sample model

In [7]:
RooRealVar mean_ctl("mean_ctl", "mean_ctl", -3, -8, 8);
RooGaussian gx_ctl("gx_ctl", "gx_ctl", x, mean_ctl, sigma);

Construct the background pdf

In [8]:
RooRealVar a0_ctl("a0_ctl", "a0_ctl", -0.1, -1, 1);
RooRealVar a1_ctl("a1_ctl", "a1_ctl", 0.5, -0.1, 1);
RooChebychev px_ctl("px_ctl", "px_ctl", x, RooArgSet(a0_ctl, a1_ctl));

Construct the composite model

In [9]:
RooRealVar f_ctl("f_ctl", "f_ctl", 0.5, 0., 1.);
RooAddPdf model_ctl("model_ctl", "model_ctl", RooArgList(gx_ctl, px_ctl), f_ctl);

Generate events for both samples
 ---------------------------------------------------------------

Generate 1000 events in x and y from model

In [10]:
RooDataSet *data = model.generate(RooArgSet(x), 100);
RooDataSet *data_ctl = model_ctl.generate(RooArgSet(x), 2000);

Create index category and join samples
 ---------------------------------------------------------------------------

Define category to distinguish physics and control samples events

In [11]:
RooCategory sample("sample", "sample");
sample.defineType("physics");
sample.defineType("control");

Construct combined dataset in (x,sample)

In [12]:
RooDataSet combData("combData", "combined data", x, Index(sample), Import("physics", *data),
                    Import("control", *data_ctl));

Construct a simultaneous pdf in (x,sample)
 -----------------------------------------------------------------------------------

Construct a simultaneous pdf using category sample as index

In [13]:
RooSimultaneous simPdf("simPdf", "simultaneous pdf", sample);

Associate model with the physics state and model_ctl with the control state

In [14]:
simPdf.addPdf(model, "physics");
simPdf.addPdf(model_ctl, "control");

Perform a simultaneous fit
 ---------------------------------------------------

Perform simultaneous fit of model to data and model_ctl to data_ctl

In [15]:
simPdf.fitTo(combData);

[#1] INFO:Minization -- RooMinimizer::optimizeConst: activating const optimization
RooAbsTestStatistic::initSimMode: creating slave calculator #0 for state physics (100 dataset entries)
RooAbsTestStatistic::initSimMode: creating slave calculator #1 for state control (2000 dataset entries)
[#1] INFO:Fitting -- RooAbsTestStatistic::initSimMode: created 2 slave calculators.
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gx,px)
[#1] INFO:Minization --  The following expressions will be evaluated in cache-and-track mode: (gx_ctl,px_ctl)
 **********
 **    1 **SET PRINT           1
 **********
 **********
 **    2 **SET NOGRAD
 **********
 PARAMETER DEFINITIONS:
    NO.   NAME         VALUE      STEP SIZE      LIMITS
     1 a0          -1.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     2 a0_ctl      -1.00000e-01  2.00000e-01   -1.00000e+00  1.00000e+00
     3 a1           4.00000e-03  2.00000e-01   -1.00000e+00  1.00000e+00
     4 a1_ct

Plot model slices on data  slices
 ----------------------------------------------------------------

Make a frame for the physics sample

In [16]:
RooPlot *frame1 = x.frame(Bins(30), Title("Physics sample"));

Plot all data tagged as physics sample

In [17]:
combData.plotOn(frame1, Cut("sample==sample::physics"));

[#1] INFO:Plotting -- RooTreeData::plotOn: plotting 100 events out of 2100 total events


Plot "physics" slice of simultaneous pdf.
 NBL You _must_ project the sample index category with data using ProjWData
 as a RooSimultaneous makes no prediction on the shape in the index category
 and can thus not be integrated

In [18]:
simPdf.plotOn(frame1, Slice(sample, "physics"), ProjWData(sample, combData));
simPdf.plotOn(frame1, Slice(sample, "physics"), Components("px"), ProjWData(sample, combData), LineStyle(kDashed));

[#1] INFO:Plotting -- RooSimultaneous::plotOn(simPdf) plot on x represents a slice in the index category (sample)
[#1] INFO:Plotting -- RooAbsReal::plotOn(model) slice variable sample was not projected anyway
[#1] INFO:Plotting -- RooSimultaneous::plotOn(simPdf) plot on x represents a slice in the index category (sample)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) directly selected PDF components: (px)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsReal::plotOn(model) slice variable sample was not projected anyway


The same plot for the control sample slice

In [19]:
RooPlot *frame2 = x.frame(Bins(30), Title("Control sample"));
combData.plotOn(frame2, Cut("sample==sample::control"));
simPdf.plotOn(frame2, Slice(sample, "control"), ProjWData(sample, combData));
simPdf.plotOn(frame2, Slice(sample, "control"), Components("px_ctl"), ProjWData(sample, combData),
              LineStyle(kDashed));

TCanvas *c = new TCanvas("rf501_simultaneouspdf", "rf403_simultaneouspdf", 800, 400);
c->Divide(2);
c->cd(1);
gPad->SetLeftMargin(0.15);
frame1->GetYaxis()->SetTitleOffset(1.4);
frame1->Draw();
c->cd(2);
gPad->SetLeftMargin(0.15);
frame2->GetYaxis()->SetTitleOffset(1.4);
frame2->Draw();

[#1] INFO:Plotting -- RooTreeData::plotOn: plotting 2000 events out of 2100 total events
[#1] INFO:Plotting -- RooSimultaneous::plotOn(simPdf) plot on x represents a slice in the index category (sample)
[#1] INFO:Plotting -- RooAbsReal::plotOn(model_ctl) slice variable sample was not projected anyway
[#1] INFO:Plotting -- RooSimultaneous::plotOn(simPdf) plot on x represents a slice in the index category (sample)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model_ctl) directly selected PDF components: (px_ctl)
[#1] INFO:Plotting -- RooAbsPdf::plotOn(model_ctl) indirectly selected PDF components: ()
[#1] INFO:Plotting -- RooAbsReal::plotOn(model_ctl) slice variable sample was not projected anyway


Draw all canvases 

In [20]:
%jsroot on
gROOT->GetListOfCanvases()->Draw()